In [1]:
import os
import sys
import time
import urllib.request
import datetime
import time
import json
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
client_id = 'your_apiclient_secret = 'your_api'

## 크롤링용 함수 정의

In [4]:
base_url = 'https://openapi.naver.com/v1/search/blog.json'

def getresult(client_id,client_secret,query,display=10,start=1,sort='sim'):
    encQuery = urllib.parse.quote(query)
    url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'

    my_request = urllib.request.Request(url)
    my_request.add_header("X-Naver-Client-Id",client_id)
    my_request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(my_request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)

    body = response_body.decode('utf-8')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    #body를 나누기
    list1 = body.split('\n\t\t{\n\t\t\t')

    #naver블로그 글만 가져오기
    list1 = [i for i in list1 if 'naver' in i]

    #블로그 제목, 링크 뽑기
    links = []
    for i in list1:
        link = re.findall('"link":"(.*?)",\n\t\t\t"description"',i)
        links.append(link)

    links = [r for i in links for r in i]
    
    # 링크를 다듬기 (필요없는 부분 제거 및 수정)
    blog_links = []
    for i in links:
        a = i.replace('\\','')
        b = a.replace('?Redirect=Log&logNo=','/')
        blog_links.append(b)

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(1)

    #블로그 링크 하나씩 불러서 크롤링
    titles = []
    contents = []
    for i in blog_links:
        #블로그 링크 하나씩 불러오기
        driver.get(i)
        time.sleep(0.2)
        try:
            #블로그 안 본문이 있는 iframe에 접근하기
            driver.switch_to.frame("mainFrame")
            #본문 내용 크롤링하기
            a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
            contents.append(a)
        # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
        except NoSuchElementException:
            contents.append('비네이버')
        #제목 전체 크롤링하기
        try:
            b = driver.find_element(By.CSS_SELECTOR,'div.pcol1').text
            titles.append(b)
        except NoSuchElementException:
            titles.append('비네이버')
    driver.quit()

    return pd.DataFrame({'제목':titles, '링크':blog_links,'내용':contents})

## 테스트용으로 돌려보시기 용이하게 n_display와 for문 리스트의 값을 변경했습니다.

In [5]:
query = '풋살장'
n_display = 3
start = 1
sort = 'sim'

# for문을 순회하면서 1100개의 데이터 수집
result_all=pd.DataFrame()
for i in [1]:
    start= i
    result= getresult(client_id,client_secret,query,display,start,sort)
    
    result_all=pd.concat([result_all,result])

C:\Users\user\AppData\Local\Temp\ipykernel_24372\1272821319.py:53: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_24372\1272821319.py:53: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_24372\1272821319.py:53: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_24372\1272821319.py:53: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_24372\1272821319.py:53: DeprecationWarning: executable_path has been deprecated, please p

In [6]:
result_all.reset_index(drop=True, inplace=True)
result_all

,제목,링크,내용
0,골때리는 그녀들 실내 풋살장 시공,https://blog.naver.com/happybobo01/222983103803,강추위가 거듭되면서 시공하기 어려운 계절에 접어들었습니다. 다행히 이번 시공은 실내...
1,"부천 중동 상동에서 찾기 힘든 어린이 풋살장, 꿈마을어린이공원",https://blog.naver.com/mudaepo1213/223114930989,저희 큰아이 축구 중독되었습니다.\n휴일 내내 이틀 동안 비가 와 축구 한번 못해보...
2,부산상가임대 대형헬스장 수영장 테니스 풋살장 실내운동시설,https://blog.naver.com/chun4112/223076914859,부산상가임대 대형헬스장 수영장 테니스 풋살장 실내운동시설\n안녕하세요.. 박동진 소...
3,난지천공원 풋살장 _ 평일 2시간 대관 후기(아이들과 놀기 좋은 곳),https://blog.naver.com/relish830/223092283890,5월입니다-! 날도 좋고 놀기도 좋은 봄날에 또…학원을 뒤로 하고 하교 후 풋살장에...
4,"직장인 여자축구, 취미활동 추천 광명 하프타임(halftime)풋살장에서 원데이클래스",https://blog.naver.com/bhj0691/223092660811,요새 술을 끊다보니 운동이나 다른 활동들을\n하게 되더라구요☺️\n그 와중에 매번 ...
...,...,...,...
1088,쌍용동 퍼펙트 풋살장에서 풋살 즐겼습니다.,https://blog.naver.com/bogers222/221299347558,비네이버
1089,풋살매칭 · 풋살장예약 아이엠그라운드 리뉴얼,https://blog.naver.com/mchan21/222776185402,풋살매칭 · 풋살장예약 아이엠그라운드 리뉴얼\n사실상 사회적 거리두기가 완전 해제 ...
1090,수원 풋살장 호매실풋살파크 대관료 알려 드려요,https://blog.naver.com/hek20/223043559681,수원 풋살장 호매실풋살파크\n야유회 및 대관진행으로\n최적화가 되어 있는곳입니다.\...
1091,대구풋살장 수성대학교 fc캐논,https://blog.naver.com/puxoty/222855156516,안녕하세요~~\n입추한지 일주일이 됬네요\n요즘 저녁이 쌀쌀 한게\n정말 가을이 온...


## 태그, 특수문자 등 제거

In [7]:
def remove_tags(text):
    clean_text = re.sub(r'(<([^>]+)>)', '', text)
    return clean_text

def remove_special_characters(text):
    clean_text = re.sub(r'[^\w\s_]', '', text)  # 언더바(_)도 포함하여 지우도록 수정
    clean_text = re.sub(r'\n', ' ', clean_text)
    return clean_text

def sub_html_special_char(my_str):
    my_str = str(my_str)
    
    p1 = re.compile('&lt;')
    p2 = re.compile('&gt;')
    p3 = re.compile('&amp;')
    p4 = re.compile('&apos;')
    p5 = re.compile('&quot;')

    result = p1.sub('\<', my_str)
    result = p2.sub('\>', result)
    result = p3.sub('\&', result)
    result = p4.sub('\'', result)
    result = p5.sub('\"', result)
    return result

def cleaning(text):
    result = sub_html_special_char(text)
    result = remove_tags(result)
    result = remove_special_characters(result)
    return result

In [8]:
Cleantitle = result_all['제목'].apply(cleaning)
Cleanbody = result_all['내용'].apply(cleaning)

In [9]:
result = pd.DataFrame({'제목':Cleantitle, '내용':Cleanbody})
result

,제목,내용
0,골때리는 그녀들 실내 풋살장 시공,강추위가 거듭되면서 시공하기 어려운 계절에 접어들었습니다 다행히 이번 시공은 실내체...
1,부천 중동 상동에서 찾기 힘든 어린이 풋살장 꿈마을어린이공원,저희 큰아이 축구 중독되었습니다 휴일 내내 이틀 동안 비가 와 축구 한번 못해보고 ...
2,부산상가임대 대형헬스장 수영장 테니스 풋살장 실내운동시설,부산상가임대 대형헬스장 수영장 테니스 풋살장 실내운동시설 안녕하세요 박동진 소장입니...
3,난지천공원 풋살장 _ 평일 2시간 대관 후기아이들과 놀기 좋은 곳,5월입니다 날도 좋고 놀기도 좋은 봄날에 또학원을 뒤로 하고 하교 후 풋살장에 다녀...
4,직장인 여자축구 취미활동 추천 광명 하프타임halftime풋살장에서 원데이클래스,요새 술을 끊다보니 운동이나 다른 활동들을 하게 되더라구요 그 와중에 매번 궁금했던...
...,...,...
1088,쌍용동 퍼펙트 풋살장에서 풋살 즐겼습니다,비네이버
1089,풋살매칭 풋살장예약 아이엠그라운드 리뉴얼,풋살매칭 풋살장예약 아이엠그라운드 리뉴얼 사실상 사회적 거리두기가 완전 해제 됨에...
1090,수원 풋살장 호매실풋살파크 대관료 알려 드려요,수원 풋살장 호매실풋살파크 야유회 및 대관진행으로 최적화가 되어 있는곳입니다 수원 ...
1091,대구풋살장 수성대학교 fc캐논,안녕하세요 입추한지 일주일이 됬네요 요즘 저녁이 쌀쌀 한게 정말 가을이 온게 실감이...


In [10]:
result.to_csv('크롤링 테스트.csv', index=False, encoding='utf-8-sig')